In [53]:
import pyndri
import itertools
from collections import Counter

relevances = { 'N', 'R', 'HR' }
rankingsOf5 = list(itertools.product(relevances, repeat=5))
pairsOfRankingsOf5 = list(itertools.product(rankingsOf5, rankingsOf5))
print(len(pairsOfRankingsOf5), "pairs of rankings")

def getContingencies (items, k, relevantDocumentCount):
    retrievedCounter = Counter(items[:k])
    TP = retrievedCounter['R'] + retrievedCounter['HR']
    FP = retrievedCounter['N']
    
    notRetrievedCounter = Counter(items[k:])
    TN = notRetrievedCounter['N']
    FN = relevantDocumentCount - TP
    
    return TP, FP, TN, FN

def getAveragePrecision (ranking, relevantDocumentCount):
    precisionsForAp = []
    for k in range(1, len(ranking)+1):
        TP, FP, TN, FN = getContingencies(ranking, k, relevantDocumentCount)
        precisionAtK = TP / (TP + FP)
        recallAtK = TP / (TP + FN)
        F1AtK = 2*precisionAtK*recallAtK
        if F1AtK > 0.0: 
            F1AtK /= precisionAtK + recallAtK
        accuracyAtK = (TP + TN)/(TP + FP + FN + TN)

        if ranking[k-1] == 'R' or ranking[k-1] == 'HR':
            # save for calculating AP later
            precisionsForAp.append(precisionAtK)
    
    averagePrecision = sum(precisionsForAp)/relevantDocumentCount
    return averagePrecision

averagePrecisionsForMapP = []
averagePrecisionsForMapE = []
for i, rankingPair in enumerate(pairsOfRankingsOf5):
    P = rankingPair[0]
    E = rankingPair[1]
    
    # show the pair
#     print ('\nP: ', P, '\nE: ', E, '\n')

    
    # implement 1 of (binary):
    #   precision at rank k
    #   recall at rank k
    #   average precision   <--
    totalCounter = Counter(P) + Counter(E)
    relevantDocumentCount = totalCounter['R'] + totalCounter['HR']
    
    if relevantDocumentCount == 0:
        # result is irrelevant
        continue
        
    averagePrecisionP = getAveragePrecision(P, relevantDocumentCount)
    averagePrecisionE = getAveragePrecision(E, relevantDocumentCount)
    
    # save for calculating MAP later
    averagePrecisionsForMapP.append(averagePrecisionP)
    averagePrecisionsForMapE.append(averagePrecisionE)

    # implement 2 of (multi-graded):
    #   nDCG at rank k
    #   ERR
    
    
    # only try a few for now
#     if i >= 10000:
#         break;
        
        
        
meanAveragePrecisionP = sum(averagePrecisionsForMapP)/len(averagePrecisionsForMapP)
meanAveragePrecisionE = sum(averagePrecisionsForMapE)/len(averagePrecisionsForMapE)
print('MAP (P) after {} results: {}'.format(len(averagePrecisionsForMapP), meanAveragePrecisionP))
print('MAP (E) after {} results: {}'.format(len(averagePrecisionsForMapE), meanAveragePrecisionE))

59049 pairs of rankings
MAP (P) after 59048 results: 0.39938612403212703
MAP (E) after 59048 results: 0.3993861240321601
